In [1]:
flex_title = "Fintech architecture"
flex_author = "Author: Kamil Marczak"
flex_orientation = "columns"
flex_include_source = False
# flex_source_link = "https://github.com/unkletam/Investment_Flow_Type_Classification"
flex_vertical_layout = "scroll"

In [2]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 10px;
</style>

In [3]:

# import diagram_gen
import ipywidgets as widgets
import yaml
from IPython.display import display
from ipywidgets import Layout

%load_ext autoreload
# Set extension to reload modules every time before executing code
%autoreload 2

# import pprint

# from diagram_gen import diagram_core_banking, diagram_kyc, diagram_lendings, diagram_2f, diagram_management, diagram_transaction

# yaml_file = open("conf.yaml")
# config_file = yaml.safe_load(yaml_file)

config_file = {'Accounts': ['Bank accounts', 'Wallets', '3Rd crypto', 'Crypto hosted'],
 'Brokerage': ['Bonds',
               'Listed shares',
               'Unlisted securities',
               'Derivatives',
               'Receivables'],
 'Cards': ['Card issuer', '3Rd party provider'],
 'Client interface': ['Web app', 'Mobile', 'Customer support'],
 'Client type': ['Private individuals', 'Business'],
 'Entity type': ['Universal banks',
                 'Investment banks',
                 'Private banking and wealth management',
                 'Collective investment fund',
                 'Insurance companies',
                 'Brokerage firms',
                 'Lending company'],
 'Inhouse or external': ['Inhouse', 'Externam', 'Mix'],
 'Lending': {'AI Scoring': 'AI Scoring',
             'No Lendings': None,
             'Traditional Scoring': 'Traditional Scoring'},
 'Localisation': ['Eea european economic area',
                  'Usa',
                  'Central-South africa',
                  'Far east',
                  'middle East',
                  'Other'],
 'Management': ['CRM',
                'Reporting - supervision',
                'Reporting - shareholders',
                'Risk Management',
                'Branches Management'],
 'Security': ['KYC', '2FA', 'AML', 'Active Fraud Prevention'],
 'Time to market': None,
 'Transactions': ['Clearing house integrations',
                  'Integration 3r party solution',
                  'Cryptocurrency internal',
                  'Cryptocurrency 3rd'],
 'White lavel provider': ['None',
                          'Transactions',
                          'Cards',
                          'Baas',
                          'Payment getaway for merchants']}

# pprint.pprint(config_file)

def printer(data):
    display(data)


def outputer(widget, method=None):
    # output = widgets.interactive_output(method, {'lendings': widget,})

    diagram = widgets.interactive_output(method, {'data': widget, })
    return diagram


def choice_outputer(widget):
    output = widgets.interactive_output(printer, {'data': widget, })
    return output

In [4]:
from IPython.display import display
from diagrams import Cluster, Diagram
from diagrams import Edge
from diagrams.aws.iot import IotBank
from diagrams.generic.compute import Rack
from diagrams.generic.device import Mobile
from diagrams.onprem.storage import CEPH_OSD
from diagrams.outscale.network import SiteToSiteVpng
from diagrams.outscale.security import IdentityAndAccessManagement, Firewall
from diagrams.programming.flowchart import Database



def diagram_core_banking(brockerage=None, client_type=None, lendings=None):

    # ('Bonds', 'Listed shares', 'Unlisted securities', 'Derivatives', 'Receivables')
    brockerage_args = brockerage
    client_type_args = client_type
    lendings_args = lendings

    with Diagram("Core Banking Solution", show=False, direction="TB") as diag_cbs:
        with Cluster("Only Write DB", ):

            database_history = Database("Database History")

            if brockerage_args:
                with Cluster("Core Banking Solution Brokerage"):
                    cbs_brokerage = [CEPH_OSD(i) for i in brockerage_args]

                    cbs_brokerage >> database_history

            if client_type_args:
                with Cluster("Core Banking Solution Clients Data"):
                    cbs_client_data = list()

                    if "Private individuals" in client_type_args:
                        cbs_client_private = CEPH_OSD("clients_private")
                        cbs_client_data += [cbs_client_private]

                    if "Business" in client_type_args:
                        cbs_client_business_repr = CEPH_OSD("clients_business_representatives")
                        cbs_client_business_pers = CEPH_OSD("clients_business_company")
                        cbs_client_data += [cbs_client_business_repr, cbs_client_business_pers]

                    if cbs_client_data:
                        database_history << cbs_client_data

            with Cluster("Core Banking Solution Current Balances"):
                deposits = CEPH_OSD("deposits")
                transactions = CEPH_OSD("transactions")

                cbs_balances = [deposits, transactions]

                database_history << cbs_balances

            if lendings_args:
                with Cluster("Core Banking Solution Loans"):
                    loans = CEPH_OSD("loans")

                    database_history << loans


        with Cluster("Only Read DB"):
            replica_master = CEPH_OSD("master")
            replica_master - [CEPH_OSD("r1"),
                              CEPH_OSD("r2"),
                              CEPH_OSD("rn")]

    display(diag_cbs)


def diagram_transaction(transactions=None):

    transactions_args = transactions

    if transactions_args:
        with Diagram("Transations", show=False, direction="TB") as diag_trans:
            with Cluster("INTERNAL"):
                with Cluster("Core Banking Solution"):
                    cbs = CEPH_OSD("Core Banking")
                with Cluster("Transaction Services"):
                    bth = CEPH_OSD("Backend Internal Handler")
                with Cluster("Frontend"):
                    fe = CEPH_OSD("Frontend Interface")
                if "Cryptocurrency internal" in transactions_args:
                    with Cluster("Blockain Ledger"):
                        cci = CEPH_OSD("Cryptocyrrency transactions_args Internal System")

            with Cluster("EXTERNAL"):
                if "Integration 3r party solution" in transactions_args:
                    tp3 = IotBank("Transaction Provider 3rd")
                    tp3 << bth >> [cbs, fe]  # transaction 3rd
                if 'Clearing house integrations' in transactions_args:
                    ch = IotBank("Clearing House")
                    cbs << bth >> ch  # clearing house integrations
                if "integration 3r party solution" in transactions_args:
                    cc3 = CEPH_OSD("Cryptocyrrency transactions_args 3rd")
                    [fe, cbs] >> cc3  # crypto transactions_args 3rd
                if "Cryptocurrency internal" in transactions_args:
                    ccb = CEPH_OSD("Cryptocyrrency Brockerage")
                    cbs << bth << cci >> ccb  # crypto transactions_args internal


        display(diag_trans)

def diagram_management(management=None):

    management_arg = management

    if management_arg:

        with Diagram("Orgaisation Management", show=False, direction="LR") as diag_manage:
            if 'Reporting - supervision' or 'Risk Management' in management_arg:
                with Cluster("EXTERNAL"):
                    fsa = IotBank("Finanacial Supervision Authority")
                    if 'Risk Management' in management_arg:
                        securities_db = CEPH_OSD("Securities public data")
            with Cluster("INTERNAL"):
                with Cluster("Core Banking Solution"):
                    cbs = CEPH_OSD("Core Banking")

                if 'CRM' in management_arg:
                    with Cluster("All microservices in project"):
                        tho  = CEPH_OSD("service")

                with Cluster("Internal Management"):
                    if 'CRM' in management_arg:
                        crm = IotBank("CRM System for internal use")
                        crm << Edge(color="firebrick", style="dashed") >> tho

                    if 'Reporting - shareholders' in management_arg:
                        ir = IotBank("Internal Reporting Module (for share holders)")
                        cbs >> Edge(color="firebrick", style="dashed") >> ir  # internal reporting

                    if 'Reporting - supervision' in management_arg:
                        sr = IotBank("Statutory Reporting Module (for FSA)")
                        cbs << sr >> fsa  #statutory supervision

                    if 'Branches Management' in management_arg:
                        with Cluster("Internal Management"):
                            bm1 = IotBank("Branches and Local Offices Management Module")
                            bm2 = IotBank("Local Branch Access")

                            bm1 << Edge(color="firebrick", style="dashed") >> bm2  # our supervision over branches and subsidiaries
                    if 'Risk Management' in management_arg:
                        with Cluster("Risk management"):
                            data_col =  CEPH_OSD("data_colector")
                            data_list = [fsa, securities_db, cbs]
                            data_col << data_list

                            risk =  Rack("compute risk exposition")
                            resolution =  Rack("Resolution scenarios")
                            stres =  Rack("Stress tests")

                            compute = [risk, resolution, stres]
                            data_col >> compute
        display(diag_manage)


def diagram_2f(security=('KYC', '2FA', 'AML')):

    securit_args = security

    if "2FA" in securit_args:
        with Diagram("Proces of obtaing second factor autentification or password retreiving", show=False,
                     direction="TB") as diag_2f:
            fe = SiteToSiteVpng("Frintent Gateway")
            with Cluster("Gateway Center"):
                udib = CEPH_OSD("User initial credential")

                sms = SiteToSiteVpng("SMS Gateway")
                email = SiteToSiteVpng("e-mail Gateway")
                voice = SiteToSiteVpng("Voice Call")

            udb = CEPH_OSD("Verification of user input with database")

            with Cluster("High Risk od Fraud"):
                fraud = Firewall("Fail")
                fraud_f = Firewall("Frud Prevention Procedures")

            with Cluster("Success Veryfication"):
                succes = IdentityAndAccessManagement("Succes")
                succes_f = IdentityAndAccessManagement("accepting further process")

            fe >> udib >> [sms, email, voice] >> udb >> [fraud, succes]
            fraud >> fraud_f
            succes >> succes_f

        display(diag_2f)


def diagram_kyc(client_type=None, security=('KYC', '2FA', 'AML')):

    client_type_args = client_type
    securit_args = security

    if "KYC" in securit_args:

        with Diagram("KYC", show=False, direction="TB") as diag_kyc:

            # if "Private individuals" in client_type_args:
            with Cluster("Person Verification - mobile app only (KNY by web is prochibited!"):
                pho = Mobile("Photo of valid document")
                doc = Mobile("Photo of valid document")

            if "Business" in client_type_args:
                with Cluster("Corporate Documents - web/mobile app"):
                    statutory = Mobile("Scans of Statutory documents")
                    representation = Mobile("Scans of representation documents")

            with Cluster("Backend"):
                val = SiteToSiteVpng("Validation of document quality")
                if "Business" in client_type_args:
                    val_corpo = SiteToSiteVpng("Validation of document quality")
                auth_r = SiteToSiteVpng("Analysis of provider response")

            auth_s = SiteToSiteVpng("Status procesing")

            with Cluster("external KYC provider"):
                auth_3rd = SiteToSiteVpng("external person KYC provider")
                if "Business" in client_type_args:
                    corpo_registry = SiteToSiteVpng("national corporate registr")

            with Cluster("Failure"):
                fail = Firewall("Fail")
                fraud_f = Firewall("Frud Prevention Procedures")

            not_f = SiteToSiteVpng("Process error -> redoo")

            with Cluster("Success Veryfication"):
                succes = IdentityAndAccessManagement("Succes")
                succes_f = IdentityAndAccessManagement("accepting further process")

            if "Business" in client_type_args:
                [statutory, representation] >> val_corpo >> corpo_registry >> auth_r

            [pho, doc] >> val >> auth_3rd >> auth_r
            auth_r >> auth_s
            auth_s >> [fail, succes]
            fail >> [fraud_f, not_f]
            succes >> succes_f

        display(diag_kyc)


# def diagram_lendings(lendings):
def diagram_lendings(lendings):

    from diagrams import Cluster, Diagram, Edge
    from diagrams.outscale.security import IdentityAndAccessManagement, Firewall
    from diagrams.onprem.storage import CEPH_OSD
    from diagrams.generic.device import Mobile

    # lending_options = lendings
    lending_options = lendings

    with Diagram("Lending", show=False, direction="TB") as diag_lending:

        with Cluster("Data Source"):
                with Cluster("Core Banking"):
                    failed_credits = CEPH_OSD("Credit history module")
                    peronal = CEPH_OSD("Personal creditee data")
                with Cluster("External"):
                    credit_history = CEPH_OSD("credit history database noted by financial supervision")
                    if lending_options == "AI Scoring":
                        alternative_source = CEPH_OSD("Alternative data source")

                data_sources = CEPH_OSD("Alternative data source")
                data_sources_node = [failed_credits, peronal, credit_history]
                if lending_options == "AI Scoring":
                    data_sources_node.append(alternative_source)

                data_sources_node >> data_sources #merging data sources

        with Cluster("Credit Performance Module"):
            with Cluster("Static Copy of Data"):
                test_data = CEPH_OSD("depersonalised data input")
                output_model = CEPH_OSD("development scoring model")
                # output_model_prod = CEPH_OSD("production scoring model")

            with Cluster("Model Computation"):
                score_engine = CEPH_OSD(
                    "Traditional Score Engine") if lending_options == "Traditional Scoring" else CEPH_OSD("AI Score Ingine")

            data_sources >> test_data # compy data to independet database
            test_data >> score_engine >> output_model
            # output_model >> Edge(label="after validation") >> output_model_prod

        with Cluster("Current credit decision module"):
            client = Mobile("Client interface")
            model = CEPH_OSD("production scoring model")
            acceptance = IdentityAndAccessManagement("decision granted")
            decline = Firewall("decision declined")
            core_banking_credit = CEPH_OSD("core banking credit tables")

            output_model >> Edge(label="periodic update") >> model
            client >> Edge(label="credit inquery") >> model
            data_sources >> model
            model >> [acceptance, decline]
            acceptance >> [core_banking_credit, client]
            decline >> client

    display(diag_lending)


In [5]:
style = {'description_width': 'initial'}

layout = widgets.Layout(width='auto', height='auto')

# items_layout = Layout(width='auto')  # override the default width of the button to 'auto' to let the button grow
items_layout = Layout(display='flex',
                      width='100%',
                      flex_flow='row',
                      justify_content='space-between')  # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    height='100%',
                    width='100%')


diagram = Layout(display='flex',
                      height='auto',
                        width='auto',)

time_to_market = widgets.BoundedIntText(value=12, min=3, description="Time to Market",
                                        layout=items_layout)
time_to_market_result = choice_outputer(time_to_market)


inhouse_out = widgets.Dropdown(options=config_file["Inhouse or external"], description='Inhouse or external',
                               disabled=False, layout=items_layout)
inhouse_out_result = choice_outputer(inhouse_out)

entity = widgets.Dropdown(options=config_file["Entity type"], description='Entity type', disabled=False,
                          layout=items_layout)
entity_results = choice_outputer(entity)
# display(entity, entity_results)

client_type = widgets.SelectMultiple(options=config_file["Client type"], description="Client type", disabled=False,
                                     layout=items_layout)


localication = widgets.Dropdown(
    options=config_file["Localisation"],
    description='Localisation',
    disabled=False,
    layout=items_layout,
)

# display(localication, localication.value)

brockerage = widgets.SelectMultiple(
    options=config_file["Brokerage"],
    description='Brokerage',
    disabled=False,
    layout=items_layout,
    continuous_update=True,
)

# brockerage_value = choice_outputer(brockerage)

# display(brockerage, brockerage_value)

transactions = widgets.SelectMultiple(
    options=config_file["Transactions"],
    description='Transactions',
    disabled=False,
    layout=items_layout,
)

# display(transactions, transactions.value)

management = widgets.SelectMultiple(
    options=config_file["Management"],
    description='Management',
    disabled=False,
    layout=items_layout,
)

# display(management, management.value)

security = widgets.SelectMultiple(
    options=config_file["Security"],
    description='Security',
    disabled=False,
    layout=items_layout,
    value=("KYC", "2FA", "AML")
)

# display(security, security.value)

cards = widgets.SelectMultiple(
    options=config_file["Cards"],
    description='Cards',
    disabled=False,
    layout=items_layout,
)

# display(cards, cards.value)

client_interface = widgets.SelectMultiple(
    options=config_file["Client interface"],
    description='Client interface',
    disabled=False,
    layout=items_layout,
)

# display(client_interface, client_interface.value)

lendings = widgets.Dropdown(
    options=config_file["Lending"],
    description='Lending',
    disabled=False,
    layout=items_layout,
)

lendings_value = choice_outputer(lendings)
# display(lendings, lendings.value)

accounts = widgets.SelectMultiple(
    options=config_file["Accounts"],
    description='Accounts',
    disabled=False,
    layout=items_layout,
)

# display(accounts, accounts.value)

white_label = widgets.SelectMultiple(
    options=config_file["White lavel provider"],
    description='White lavel provider',
    disabled=False,
    layout=items_layout,
)

# display(white_label, white_label.value)

## Column 1

### Tab 1

In [6]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

items = [time_to_market, entity, client_type, localication, brockerage, transactions, management, security, cards,
         client_interface,
         lendings, accounts, white_label]

box = widgets.Box(children=items, layout=box_layout)
display(box)

Box(children=(BoundedIntText(value=12, description='Time to Market', layout=Layout(display='flex', flex_flow='…

### Reload

In [7]:
from IPython.display import Javascript, display
items_layout = Layout( width='100%', height='100%', )     # override the default width of the button to 'auto' to let the button grow


def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))


out = widgets.Output(layout={'border': '1px solid black'})
with out:
    button = widgets.Button(description="Create next input", layout=items_layout, button_style='danger')
    button.on_click(run_all)
    display(button)
display(out)

Output(layout=Layout(border='1px solid black'))

## Column 2

### core banking solution

In [8]:
# out = widgets.Output(layout={'border': '1px solid black'})

# def update_core_banking(args):
#     with out:
#         d_cbs = widgets.interactive_output(diagram_core_banking, {'brockerage': brockerage, 'client_type': client_type, 'lendings': lendings})
#         display(d_cbs)



# brockerage.observe(update_core_banking, 'value')
# client_type.observe(update_core_banking, 'value')
# lendings.observe(update_core_banking, 'value')


# d_cbs = widgets.interactive_output(diagram_core_banking, {'brockerage': brockerage, 'client_type': client_type, 'lendings': lendings})
# display(d_cbs)

In [9]:
# update_core_banking(None)
# display(out)

d_cbs = widgets.interactive_output(diagram_core_banking, {'brockerage': brockerage, 'client_type': client_type, 'lendings': lendings})
display(d_cbs)

Output()

### Lendings

In [10]:
d_len =  widgets.interactive_output(diagram_lendings, {'lendings': lendings})
display(d_len)

Output()

### KYC

In [11]:
d_kyc = widgets.interactive_output(diagram_kyc, {'client_type': client_type, "security": security})

display(d_kyc)

Output()